In [1]:
#Gather relevant keys from our Secret Scope
ServicePrincipalID = dbutils.secrets.get(scope = "Analysts", key = "SPID")
ServicePrincipalKey = dbutils.secrets.get(scope = "Analysts", key = "SPKey")
DirectoryID = dbutils.secrets.get(scope = "Analysts", key = "DirectoryID")
DBUser = dbutils.secrets.get(scope = "Analysts", key = "DBUser")
DBPassword = dbutils.secrets.get(scope = "Analysts", key = "DBPword")


#Combine DirectoryID into full string
Directory = "https://login.microsoftonline.com/{}/oauth2/token".format(DirectoryID)

#Configure our ADLS Gen 2 connection with our service principal details
spark.conf.set("fs.azure.account.auth.type", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id", ServicePrincipalID)
spark.conf.set("fs.azure.account.oauth2.client.secret", ServicePrincipalKey)
spark.conf.set("fs.azure.account.oauth2.client.endpoint", Directory)

### Working with dirty data
In this example, we're going to look at a file that doesn't necessarily fit our expected schema. We will look at methods of rejecting rows and building resilient pipelines.

We are expecting the data to conform to a specified schema, so we'll use a hardcoded schema and see how this can be applied to our dubious dataset.

There are three different modes for parsing text files:
 - PERMISSIVE - Allow all rows through but nullify all columns for malformed rows
 - FAILFAST - Error if any row cannot be parsed
 - DROPMALFORMED - Drop any rows that cannot be parsed from the dataset

In [3]:
# Bring in the libraries needed to read JSON and convert to StructTypes
import json
from pyspark.sql.types import *

mySchema = StructType([
  StructField("LocationID", IntegerType(), True),
  StructField("Borough", StringType(), True),
  StructField("Zone", StringType(), True),
  StructField("service_zone", StringType(), True)])

# Now let's load the data, allowing any malformed records to come through
df = (spark
       .read
       .option("header","true")
       .option("mode", "PERMISSIVE")
       .schema(mySchema)
       .csv("abfss://root@dblake.dfs.core.windows.net/RAW/Public/TaxiZones/V1/taxi+_zone_lookup.csv")
     )

# If we look at the results, everything seems fine - but we might spot a NULL row or two in there
display(df.select("LocationID", "Borough", "Zone", "service_zone"))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


In [4]:
# Now let's load the data, allowing any malformed records to come through
failfastdf = (spark
       .read
       .option("header","true")
       .option("mode", "FAILFAST")
       .schema(mySchema)
       .csv("abfss://root@dblake.dfs.core.windows.net/RAW/Public/TaxiZones/V1/taxi+_zone_lookup.csv")
     )

# This succeeds - all we have done is defined a transform, not read the data, so it hasn't validated our schema yet

In [5]:
# If we try and view the results, we'll get an error - as soon as something cannot parse, it gives up
display(failfastdf.select("LocationID", "Borough", "Zone", "service_zone"))

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 92.0 failed 4 times, most recent failure: Lost task 0.3 in stage 92.0 (TID 1199, 10.139.64.5, executor 1): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST.
	at org.apache.spark.sql.execution.datasources.FailureSafeParser.parse(FailureSafeParser.scala:82)
	at org.apache.spark.sql.execution.datasources.csv.UnivocityParser$$anonfun$parseIterator$2.apply(UnivocityParser.scala:361)
	at org.apache.spark.sql.execution.datasources.csv.UnivocityParser$$anonfun$parseIterator$2.apply(UnivocityParser.scala:361)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:434)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:228)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:196)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:622)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:51)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:148)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:147)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:139)
	at org.apache.spark.scheduler.Task.run(Task.scala:112)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:497)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1432)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:503)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.RuntimeException: Malformed CSV record
	at org.apache.spark.sql.execution.datasources.csv.UnivocityParser.org$apache$spark$sql$execution$datasources$csv$UnivocityParser$$convert(UnivocityParser.scala:249)
	at org.apache.spark.sql.execution.datasources.csv.UnivocityParser.parse(UnivocityParser.scala:214)
	at org.apache.spark.sql.execution.datasources.csv.UnivocityParser$$anonfun$7.apply(UnivocityParser.scala:353)
	at org.apache.spark.sql.execution.datasources.csv.UnivocityParser$$anonfun$7.apply(UnivocityParser.scala:353)
	at org.apache.spark.sql.execution.datasources.FailureSafeParser.parse(FailureSafeParser.scala:70)
	... 24 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2100)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2088)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2087)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2087)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1076)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1076)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.scheduler.DAGSchedul

In [6]:
# In this occurrence, there are rows that don't fit the structure (ie: have extra columns), so we can't even run a dataframe-wide aggregation:
failfastdf.count()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1223194938973871> in <module> () 
 1 #It's important to note that this is only when we actually need to touch the offending columns. If we can perform aggregations on the dataset as a whole, we won't see the failure 
 ----> 2 failfastdf . count ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in count (self) 
 521 2 
 522 """
 --> 523 return int ( self . _jdf . count ( ) ) 
 524 
 525 @ ignore_unicode_prefix 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o720.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 22, 10.139.64.4, executor 0): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST.
	at org.apache.spark.sql.execution.datasources.FailureSafeParser.parse(FailureSafeParser.scala:82)
	at org.apache.spark.sql.execution.datasources.csv.UnivocityParser$$anonfun$parseIterator$2.apply(UnivocityParser.scala:361)
	at org.apache.spark.sql.execution.datasources.csv.UnivocityParser$$anonfun$parseIterator$2.apply(UnivocityParser.scala:361)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:434)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:228)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:196)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:622)
	at org.apache.spark.sql.execution.columnar.CachedRDDBuilder$$anonfun$1$$anon$1.next(InMemoryRelation.scala:100)
	at org.apache.spark.sql.execution.columnar.CachedRDDBuilder$$anonfun$1$$anon$1.next(InMemoryRelation.scala:92)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:222)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:299)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:1170)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:1161)
	at org.apache.spark.storage.BlockManager.doPut(BlockManager.scala:1096)
	at org.apache.spark.storage.BlockManager.doPutIterator(BlockManager.scala:1161)
	at org.apache.spark.storage.BlockManager.getOrElseUpdate(BlockManager.scala:883)
	at org.apache.spark.rdd.RDD.getOrCompute(RDD.scala:351)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:302)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitions

Failfast has it's uses. Many ETL developers prefer this approach - having the whole data pipeline shutdown because there is a malformed file means we can be very strict about data typing & ensure that no bad data ever makes it to downstream systems. However, this does mean that we then need to implement further systems to handle these errors, maybe even having a team to support the process when it breaks

It is more common these days that we allow processing to continue, but do something with the bad data. This means we will get data to people as soon as possible, but there might be caveats associated with the data delivered

In [8]:
# Now let's load the data, allowing any malformed records to come through
dropMaldf = (spark
       .read
       .option("header","true")
       .option("mode", "DROPMALFORMED")
       .schema(mySchema)
       .csv("abfss://root@dblake.dfs.core.windows.net/RAW/Public/TaxiZones/V1/taxi+_zone_lookup.csv")
     )

# Let's take a look at the data again
display(dropMaldf.select("LocationID", "Borough", "Zone", "service_zone"))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


In [9]:
# If we force the dataframe to perform a count, it will not try to conform rows to the schema and will return the full record count
dropMaldf.count()

# You should see Out[]: 280 as a result

Out[ 7 ]: 280

In [10]:
# If we then force a cache, it will validate the schema and drop any rows, so we will see our overall count go down as malformed rows are dropped from the dataframe
dropMaldf.cache()
dropMaldf.count()

# Out[]: 274

Out[ 8 ]: 274

This is fairly simple - our DataFrame is populated and we know the data fits properly... but most data pipelines can't accept the loss of data, no matter what the problem was. So we need to look at syphoning off those malformed rows into a separate dataset

## Rejecting Failed Rows

The above methods are all well and good - but all have their drawbacks when building an ETL pipeline. A much more common approach is to push data that does not parse correctly down a different path.

Let's try it again with the PERMISSIVE option, except this time we will add in a system field "_corrupt_record". This is specifically with the PERMISSIVE mode to store the original contents of any fields that did not properly conform

In [13]:
from pyspark.sql.functions import *

mySchema = StructType([
  StructField("LocationID", IntegerType(), True),
  StructField("Borough", StringType(), True),
  StructField("Zone", StringType(), True),
  StructField("service_zone", StringType(), True),
  StructField("_corrupt_record", StringType(), True)])

# Now create our DataFrame over the whole fileset, but applying our the schema from the sample rather than inferring
Reddf = (spark
       .read
       .option("header","true")
       .option("mode", "PERMISSIVE")
       .schema(mySchema)4
       .csv("abfss://root@dblake.dfs.core.windows.net/RAW/Public/TaxiZones/V1/taxi+_zone_lookup.csv")
     )

display(Reddf.select("*"))

LocationID,Borough,Zone,service_zone,_corrupt_record
1,EWR,Newark Airport,EWR,null
2,Queens,Jamaica Bay,Boro Zone,null
3,Bronx,Allerton/Pelham Gardens,Boro Zone,null
4,Manhattan,Alphabet City,Yellow Zone,null
5,Staten Island,Arden Heights,Boro Zone,null
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone,null
7,Queens,Astoria,Boro Zone,null
8,Queens,Astoria Park,Boro Zone,null
9,Queens,Auburndale,Boro Zone,null
10,Queens,Baisley Park,Boro Zone,null


In [14]:
# We can filter using the _corrupt_record column to find only those rows which failed parsing in some manner
display(Reddf.select("*").filter(col("_corrupt_record").isNotNull()))

LocationID,Borough,Zone,service_zone,_corrupt_record
12,Manhattan,Battery Park,Yellow,"12,Manhattan,Battery Park,Yellow, Zone"
23,Staten Island,Bloomfield/Emerson Hill,Boro,"23,Staten Island,Bloomfield/Emerson Hill,Boro, Zone"
39,Brooklyn,Canarsie,Boro,"39,Brooklyn,Canarsie,Boro, Zone"
168,Bronx,Mott Haven/Port Morris,Boro,"168,Bronx,Mott Haven/Port Morris,Boro, Zone"
178,Brooklyn,Ocean Parkway South,Boro,"178,Brooklyn,Ocean Parkway South,Boro, Zone"
null,null,null,null,"N/A,N/A,N/A,N/A"


In this example we have two different types of parse failure happening

There are some rows that had an extra column - in this cases we've managed to fill our DataFrame attributes and the additional data has been left out. This assumes that the additional field was appended to the end of the record, which isn't always the case! Note - DROPMALFORMED removed all of these rows, even though they had some valid data.

The other type is an attribute parsing failure - our LocationID expects an integer but received "N/A". In this case, PERMISSIVE mode has replaced ALL columns with null, not just the offending attribute.

We can use the logic above in our data preparation pipelines. We could divert these rows to a separate fileset within the Lake for a later process to review, or allow the ones with some population through and raise an issue for someone to check at a later date. This all depends on how resilient & fault tolerant we want our pipeline to be

In [16]:
# Our data pipeline to continue for further processing
goodRowsdf = Reddf.select("*").filter(col("_corrupt_record").isNull())

# Our rejected rows to be review at a later date
rejectRowsdf = Reddf.select("*").filter(col("_corrupt_record").isNotNull())